In [1]:
def filesorter(source, destination):
    from shutil import move as sh_move # 'move' is such a simple name I'm worried it will conflict with something
    from datetime import datetime
    from errno import ENOENT
    #
    subdirs_list = [item for item in os.listdir(destination) if os.path.isdir(os.path.join(destination, item))]
    for item in subdirs_list[::-1]:
        if 'Downloads -' in item:
            subdir = 'Downloads - ' + str(int(item[12:14]) + 1) + datetime.today().strftime(' - %d.%m.%y - UNSORTED')
            break
    dir_types = {
        'Pictures': ('.jpg', '.jpeg', '.png', '.tga', '.bmp', '.webp'),
        'GIFs and videos': ('.gif', '.webm', '.mp4', '.mkv', '.avi'),
        'Executables': ('.exe', '.msi', '.msu', '.apk'),
        'Archives': ('.zip', '.rar', '.7z', '.tar'),
        'Documents': ('.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx', '.pdf', '.csv', '.tif', '.rtf'),
        'Books': ('.fb2', '.ebup', '.mobi', '.djvu'),
        'Torrent files': ('.torrent',),
        'WTFs': tuple()
    }
    # Check codepage
    # Get list of files
    files_list = [item for item in os.listdir(source) if os.path.isfile(os.path.join(source, item))]
    for file in files_list:
        _, ext = os.path.splitext(file)
        # for dir: if extension in dir: move file to dir
        for dir in dir_types:
            if ext in dir_types[dir] or dir == 'WTFs': # If we got to WTFs, the extension doesn't fit into any other type
                pass
                try:
                    sh_move(os.path.join(source, file), os.path.join(destination, subdir, dir, file))
                except IOError as e:                    
                    if e.errno != ENOENT: # Checking that this is an ENOENT(2) "No such file or directory": appears when the path hasn't been yet created
                        raise
                    os.makedirs(os.path.join(destination, subdir, dir))
                    sh_move(os.path.join(source, file), os.path.join(destination, subdir, dir, file))
                break

In [5]:
# GUI execution
from PyQt5 import QtWidgets, QtCore
import os

class widget_filesorter(QtWidgets.QWidget):
    def __init__(self):
        super(widget_filesorter, self).__init__()
        if os.path.exists(os.path.join(os.getcwd(), "Filesorter.ini")):
            f_ini = open(os.path.join(os.getcwd(), "Filesorter.ini"), 'r')
            self.source = f_ini.readline().rstrip('\n')
            self.destination = f_ini.readline().rstrip('\n')
            f_ini.close()
        else:
            self.source = os.path.join("c:\\", "Users", "SirNotAWholeLot", "Downloads")
            self.destination =  os.path.join("k:\\", "Archive", "_Data", "Downloads")
        #
        layout = QtWidgets.QVBoxLayout(self)
        label_source = QtWidgets.QLabel("Source directory:", self)
        self.box_source = QtWidgets.QLineEdit(self.source, self)
        label_destination = QtWidgets.QLabel("Destination directory:", self)
        self.box_destination = QtWidgets.QLineEdit(self.destination, self)
        button_save = QtWidgets.QPushButton("Save paths", self)
        button_save.clicked.connect(self.save_paths)
        button_sort = QtWidgets.QPushButton("Sort files", self)
        button_sort.clicked.connect(self.sort_files)
        for item in (label_source, self.box_source, label_destination, self.box_destination, button_save, button_sort): layout.addWidget(item)
        layout.addSpacerItem(QtWidgets.QSpacerItem(300, 0, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Expanding))
        
    def save_paths(self):
        source = self.box_source.text()
        destination = self.box_destination.text()
        if os.path.exists(source) and os.path.exists(destination):
            f_ini = open(os.path.join(os.getcwd(), "Filesorter.ini"), 'w')
            f_ini.write(source + '\n' + destination)
            f_ini.close()

    def sort_files(self):
        source = self.box_source.text()
        destination = self.box_destination.text()
        if os.path.exists(source) and os.path.exists(destination): filesorter(source, destination)

class this_main_window(QtWidgets.QMainWindow):
    def __init__(self):
        super(this_main_window, self).__init__()
        main_widget = widget_filesorter()
        self.setCentralWidget(main_widget)
        self.setWindowTitle("File (downloads) sorter")
        #self.setGeometry(50, 100, 750, 600)
    
if __name__ == '__main__':
    import sys
    # Modified version of the runner code for Jupyter compatibility
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    window = this_main_window()
    window.show()
    app.exec_()
    del app